In [1]:
import pandas as pd
import numpy as np
import pydeck as pdk
import random
import shapefile
import json
import math

import geopy.distance

from tqdm import tqdm

from multiprocessing import Pool
from IPython.display import clear_output

In [2]:
def read_shapefile(sf_shape):
    """
    Read a shapefile into a Pandas dataframe with a 'coords' 
    column holding the geometry information. This uses the pyshp
    package
    """

    fields = [x[0] for x in sf_shape.fields][1:]
    records = [y[:] for y in sf_shape.records()]
    #records = sf_shape.records()
    shps = [s.points for s in sf_shape.shapes()]
    df = pd.DataFrame(columns=fields, data=records)
    df = df.assign(coords=shps)
    return df


In [3]:
#загружаем информацию о сеточном разбиении МО в память
myshp = open("fishnet2021.shp", "rb")
mydbf = open("fishnet2021.dbf", "rb")
r = shapefile.Reader(shp=myshp, dbf=mydbf)
s_df = read_shapefile(r)
#в поле coords первая координата центр квадрата, остальные его углы
s_df.head()

,cell_zid,coords
0,0,"[(38.64720691553791, 54.255243332408604), (38...."
1,1,"[(38.65487820205851, 54.255170084754035), (38...."
2,2,"[(38.66254945426432, 54.255096350709195), (38...."
3,3,"[(38.670220672771414, 54.25502212698049), (38...."
4,4,"[(38.67789185552521, 54.25494741523556), (38.6..."


In [4]:
#считываем информацию о переходах Дом-Работа и прикручиваем туда координаты переходов
h_w_matrix = pd.read_csv("04_CMatrix_Home_Work_July.csv")
h_w_matrix = h_w_matrix.merge(right=s_df, how='inner', left_on='home_zid', right_on='cell_zid')
h_w_matrix = h_w_matrix.merge(right=s_df, how='inner', left_on='work_zid', right_on='cell_zid')
h_w_matrix.drop('cell_zid_x', axis=1, inplace=True)
h_w_matrix.drop('cell_zid_y', axis=1, inplace=True)

h_w_matrix.head()

,home_zid,work_zid,customers_cnt,coords_x,coords_y
0,33617,35383,3,"[(37.04909381713915, 55.14932918598869), (37.0...","[(37.13516508529589, 55.172076772159485), (37...."
1,33620,33621,2,"[(37.07262354156165, 55.149413945205254), (37....","[(37.08046681019148, 55.14944118799408), (37.0..."
2,33621,33621,2,"[(37.08046681019148, 55.14944118799408), (37.0...","[(37.08046681019148, 55.14944118799408), (37.0..."
3,33622,38813,1,"[(37.08831009005705, 55.14946792727634), (37.0...","[(37.01696105787737, 55.21209316621927), (37.0..."
4,38412,38813,1,"[(37.04057941677159, 55.207692480896384), (37....","[(37.01696105787737, 55.21209316621927), (37.0..."


In [5]:
#считываем наименование всех регинов и прикручиваем к ним их координаты
#loc_names = pd.read_excel('names.xlsx')
#loc_names['cell_zid'] = loc_names['cell_zid'].map(int)
#loc_names['adm_zid'] = loc_names['adm_zid'].map(int)
#loc_names = loc_names.merge(right=s_df, how='inner', left_on='cell_zid', right_on='cell_zid')
loc_names = pd.read_csv('rebuilded_names.csv')
loc_names.head()

,Unnamed: 0,cell_zid,area_peresechenia_s_admzone_kv.km,adm_zid,adm_name,okrug_name,sub_ter
0,0,32909,0.407116,216,Роговское,Троицкий административный округ,Новая Москва
1,1,32910,1.003458,216,Роговское,Троицкий административный округ,Новая Москва
2,2,33261,0.233312,216,Роговское,Троицкий административный округ,Новая Москва
3,3,33262,1.868032,216,Роговское,Троицкий административный округ,Новая Москва
4,4,33263,2.500630,216,Роговское,Троицкий административный округ,Новая Москва


In [6]:
#считываем информацию о плотности проживающего, работающего и проходящего населения для каждого квадрата и прикручиваем туда координаты квадратов
c_locations = pd.read_csv("01_CLocation_July.csv")
c_locations = c_locations.merge(right=s_df, how='inner', left_on='zid', right_on='cell_zid')
c_locations.drop('cell_zid', axis=1, inplace=True)
#c_locations['coords'] = c_locations['coords'].apply(lambda x: (x[0][1],x[0][0] )) # оставляем только 1 координату
c_locations['lat'] = c_locations['coords'].map(lambda x: x[0][1]) # извлекаем широту
c_locations['lon'] = c_locations['coords'].map(lambda x: x[0][0]) # извлекаем долготу

c_locations.drop('coords', axis=1, inplace=True)


c_locations.head()

,zid,customers_cnt_home,customers_cnt_job,customers_cnt_day,customers_cnt_move,lat,lon
0,32909,4,0,0,0,55.140287,37.033512
1,32910,25,8,26,0,55.140316,37.041354
2,33261,2,0,2,0,55.144748,37.025618
3,33262,13,2,11,0,55.144778,37.033460
4,33263,10,0,3,0,55.144808,37.041302


In [7]:
adm_names = loc_names['adm_name'].drop_duplicates()
print("Кол-во уникальных райнов: ",adm_names.shape[0])

Кол-во уникальных райнов:  146


### Следующий набор блоков предназначен для расчёта относительно предыдущего месяца


Сгенерируем псевдоданные за предыдущий месяц:

In [8]:
#Изменяем исходный датасет июля на рандомные % чтобы сгененрировать динамику плотности населения
c_locations_june = c_locations.copy()
c_locations_june['customers_cnt_home'] = c_locations_june['customers_cnt_home'].apply(lambda x: int(float(x) *( random.uniform(0.5, 1.7))))
c_locations_june['customers_cnt_job'] = c_locations_june['customers_cnt_job'].apply(lambda x: int(float(x) *( random.uniform(0.8, 1.1))))
c_locations_june['customers_cnt_day'] = c_locations_june['customers_cnt_day'].apply(lambda x: int(float(x) *( random.uniform(0.8, 1.1))))
c_locations_june['customers_cnt_move'] = c_locations_june['customers_cnt_move'].apply(lambda x: int(float(x) *( random.uniform(0.8, 1.1))))

c_locations_june.head()

,zid,customers_cnt_home,customers_cnt_job,customers_cnt_day,customers_cnt_move,lat,lon
0,32909,5,0,0,0,55.140287,37.033512
1,32910,33,8,25,0,55.140316,37.041354
2,33261,1,0,1,0,55.144748,37.025618
3,33262,10,2,8,0,55.144778,37.033460
4,33263,12,0,2,0,55.144808,37.041302


In [10]:
delta_cnt_df = c_locations_june.copy()
delta_cnt_df = delta_cnt_df.merge(right=c_locations, how='inner', left_on='zid', right_on='zid')
delta_cnt_df.drop(['lat_y', 'lon_y'], axis=1, inplace=True)
delta_cnt_df.rename({'lat_x':'lat', 'lon_x':'lon'}, axis=1, inplace=True)

delta_cnt_df.head()

,zid,customers_cnt_home_x,customers_cnt_job_x,customers_cnt_day_x,customers_cnt_move_x,lat,lon,customers_cnt_home_y,customers_cnt_job_y,customers_cnt_day_y,customers_cnt_move_y
0,32909,5,0,0,0,55.140287,37.033512,4,0,0,0
1,32910,15,8,28,0,55.140316,37.041354,25,8,26,0
2,33261,2,0,1,0,55.144748,37.025618,2,0,2,0
3,33262,21,1,9,0,55.144778,37.033460,13,2,11,0
4,33263,9,0,2,0,55.144808,37.041302,10,0,3,0


In [11]:
#расчёт дельты изменения плотностей
for feature_type in ['home','job','day', 'move']:
    delta_cnt_df[f'customers_cnt_{feature_type}_y'] = delta_cnt_df[f'customers_cnt_{feature_type}_x'] - delta_cnt_df[f'customers_cnt_{feature_type}_y']
    delta_cnt_df.rename({f'customers_cnt_{feature_type}_y':f'customers_dlt_{feature_type}'},axis=1, inplace=True)
    delta_cnt_df.rename({f'customers_cnt_{feature_type}_x':f'customers_cnt_{feature_type}'},axis=1, inplace=True)

delta_cnt_df.head()

,zid,customers_cnt_home,customers_cnt_job,customers_cnt_day,customers_cnt_move,lat,lon,customers_dlt_home,customers_dlt_job,customers_dlt_day,customers_dlt_move
0,32909,5,0,0,0,55.140287,37.033512,1,0,0,0
1,32910,15,8,28,0,55.140316,37.041354,-10,0,2,0
2,33261,2,0,1,0,55.144748,37.025618,0,0,-1,0
3,33262,21,1,9,0,55.144778,37.033460,8,-1,-2,0
4,33263,9,0,2,0,55.144808,37.041302,-1,0,-1,0


In [ ]:
for feature in ['home','job','day', 'move']:
    delta_cnt_df[f'predicted_customers_cnt_{feature}'] = delta_cnt_df[f'customers_cnt_{feature}'] + delta_cnt_df[f'customers_dlt_{feature}']


summ_columns = ['predicted_customers_cnt_home','predicted_customers_cnt_job','predicted_customers_cnt_day'] #поля по которым будет осуществляться сумма плотности людей
tqdm.pandas(desc="calc people flow to mfc's")

mfc_df['future_people_flow_rate'] = mfc_df['global_id'].progress_apply(lambda x: delta_cnt_df.loc[delta_cnt_df['nearest_mfc_id'] == x][summ_columns].values.sum())

delta_cnt_df.head()

In [12]:
#Расчёт логистики районов
tqdm.pandas(desc="find logistic!")
delta_cnt_df['logistic'] = 0
delta_cnt_df['logistic'] = delta_cnt_df['zid'].progress_apply(lambda x: sum(h_w_matrix.loc[h_w_matrix['work_zid'] == x]['customers_cnt'].values) + sum(h_w_matrix.loc[h_w_matrix['home_zid'] == x]['customers_cnt'].values))


C:\ProgramData\Anaconda3\lib\site-packages\tqdm\std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
find logistic!: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 10240/10240 [00:49<00:00, 207.71it/s]


In [14]:
delta_cnt_df.head()

,zid,customers_cnt_home,customers_cnt_job,customers_cnt_day,customers_cnt_move,lat,lon,customers_dlt_home,customers_dlt_job,customers_dlt_day,customers_dlt_move,logistic
0,32909,4,0,0,0,55.140287,37.033512,0,0,0,0,0
1,32910,37,8,24,0,55.140316,37.041354,12,0,-2,0,0
2,33261,3,0,1,0,55.144748,37.025618,1,0,-1,0,0
3,33262,21,1,9,0,55.144778,37.033460,8,-1,-2,0,2
4,33263,12,0,3,0,55.144808,37.041302,2,0,0,0,0


In [31]:
mfc_df = pd.read_excel('mos_coords.xlsx')
mfc_df['geodata_center'] = mfc_df['geodata_center'].apply(lambda x: (json.loads(x)['coordinates'][1],json.loads(x)['coordinates'][0])  )
mfc_df['lon'] = mfc_df['geodata_center'].apply(lambda x: x[0])
mfc_df['lat'] = mfc_df['geodata_center'].apply(lambda x: x[1])
#mfc_df['ShortName', 'WindowCount', 'lon', 'lat', 'District'].to_csv('mos_coords.csv')

save_columns = ['global_id','FullName', 'INN', 'KPP', 'Address', 'ChiefName', 'PublicPhone', 'OpenDate',  'ShortName', 'WindowCount', 'geodata_center', 'lon', 'lat', 'District']

mfc_new = pd.DataFrame(columns= save_columns)#['FullName', 'INN', 'KPP', 'Address', 'ChiefName', 'PublicPhone', 'OpenDate',  'ShortName', 'WindowCount', 'lon', 'lat', 'District'])

mfc_df = mfc_df[save_columns]

mfc_df.info() 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139 entries, 0 to 138
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   global_id       139 non-null    int64  
 1   FullName        139 non-null    object 
 2   INN             139 non-null    int64  
 3   KPP             139 non-null    int64  
 4   Address         139 non-null    object 
 5   ChiefName       139 non-null    object 
 6   PublicPhone     139 non-null    object 
 7   OpenDate        139 non-null    object 
 8   ShortName       139 non-null    object 
 9   WindowCount     139 non-null    int64  
 10  geodata_center  139 non-null    object 
 11  lon             139 non-null    float64
 12  lat             139 non-null    float64
 13  District        139 non-null    object 
 14  metaInfo        139 non-null    object 
dtypes: float64(2), int64(4), object(9)
memory usage: 16.4+ KB


In [49]:

tqdm.pandas(desc="find nearest mfc")
delta_cnt_df['nearest_mfc_id'] = delta_cnt_df['zid'].progress_apply(
    lambda x: mfc_df.loc[mfc_df['geodata_center'].apply(
        lambda y: geopy.distance.distance((y[0],y[1]),
                          (delta_cnt_df.loc[delta_cnt_df['zid']==x]['lat'].values[0],
                           delta_cnt_df.loc[delta_cnt_df['zid']==x]['lon'].values[0])
                                    ).m
    ).idxmin()]['global_id'])

print('Поиск близжайших мфц закончен')


C:\ProgramData\Anaconda3\lib\site-packages\tqdm\std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
find nearest mfc: 100%|██████████████████████████████████████████████████████████████████████████████████████████████| 10240/10240 [31:14<00:00,  5.46it/s]

Поиск близжайших мфц закончен


In [50]:


tqdm.pandas(desc="find distance to nearest mfc")
delta_cnt_df['nearest_mfc_distance'] = 0
delta_cnt_df['nearest_mfc_distance'] = delta_cnt_df['zid'].progress_apply(
    lambda x: geopy.distance.distance(mfc_df.loc[mfc_df['global_id'] == delta_cnt_df.loc[delta_cnt_df['zid']==x]['nearest_mfc_id'].values[0]]['geodata_center'], 
                                    (delta_cnt_df.loc[delta_cnt_df['zid']==x]['lat'].values[0], delta_cnt_df.loc[delta_cnt_df['zid']==x]['lon'].values[0])).m)

print('Расчёт дистаций(метр) от ячейки до мфц закончен')   


C:\ProgramData\Anaconda3\lib\site-packages\tqdm\std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
find distance to nearest mfc: 100%|█████████████████████████████████████████████████████████████████████████████████| 10240/10240 [00:24<00:00, 414.94it/s]


Расчёт дистаций(метр) от ячейки до мфц закончен


In [51]:

summ_columns = ['customers_cnt_home','customers_cnt_job','customers_cnt_day'] #поля по которым будет осуществляться сумма плотности людей
tqdm.pandas(desc="calc people flow to mfc's")

mfc_df['people_flow_rate'] = mfc_df['global_id'].progress_apply(lambda x: delta_cnt_df.loc[delta_cnt_df['nearest_mfc_id'] == x][summ_columns].values.sum())

delta_cnt_df.head()

C:\ProgramData\Anaconda3\lib\site-packages\tqdm\std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
calc people flow to mfc's: 100%|████████████████████████████████████████████████████████████████████████████████████████| 139/139 [00:00<00:00, 963.72it/s]


,Unnamed: 0,Unnamed: 0.1,zid,customers_cnt_home,customers_cnt_job,customers_cnt_day,customers_cnt_move,lat,lon,customers_dlt_home,customers_dlt_job,customers_dlt_day,customers_dlt_move,nearest_mfc,logistic,mfc_chance,nearest_mfc_id,nearest_mfc_distance
0,0,0,32909,2,0,0,0,55.140287,37.033512,-2,0,0,0,43617.483646,0,11.367988,272644507,43617.483646
1,1,1,32910,13,6,26,0,55.140316,37.041354,-12,-2,0,0,43407.995189,0,11.326988,272644507,43407.995189
2,2,2,33261,2,0,2,0,55.144748,37.025618,0,0,0,0,43381.461499,0,11.312600,272644507,43381.461499
3,3,3,33262,13,1,11,0,55.144778,37.033460,0,-1,0,0,43165.034942,2,11.266504,272644507,43165.034942
4,4,4,33263,7,0,2,0,55.144808,37.041302,-3,0,-1,0,42953.339821,0,11.211549,272644507,42953.339821


In [48]:
people_to_one_window = 3000

mfc_df['max_people_flow'] = mfc_df['WindowCount'] * people_to_one_window 

mfc_df['is_deal'] = mfc_df['max_people_flow'] >= mfc_df['people_flow_rate']

mfc_df.to_csv("mos_coords.csv")
mfc_df.head()

,global_id,FullName,INN,KPP,Address,ChiefName,PublicPhone,OpenDate,ShortName,WindowCount,geodata_center,lon,lat,District,metaInfo,max_people_flow,people_flow_rate,is_deal
0,3066058,филиал Государственного бюджетного учреждения ...,7731419456,771001001,"город Москва, Верхняя Красносельская улица, до...",Есина Ольга Викторовна,PublicPhone:(495) 777-77-77\n\n,25.03.2013,МФЦ района Красносельский,54,"(55.785871804304, 37.660717037156)",55.785872,37.660717,Красносельский район,Краткое название: МФЦ района Красносельский<br...,162000,160832,True
1,3066059,филиал Государственного бюджетного учреждения ...,7731419456,771001001,"город Москва, Чапаевский переулок, дом 16",Фурцева Ольга Николаевна,PublicPhone:(495) 777-77-77\n\n,06.05.2013,МФЦ района Сокол,41,"(55.797147273338, 37.518206847939)",55.797147,37.518207,район Сокол,Краткое название: МФЦ района Сокол<br/>Адрес у...,123000,94062,True
2,3066060,филиал Государственного бюджетного учреждения ...,7731419456,771001001,"город Москва, Новопеределкинская улица, дом 12А",Орлова Ксения Сергеевна,PublicPhone:(495) 777-77-77\n\n,03.06.2013,МФЦ района Ново-Переделкино,37,"(55.641319194391, 37.366429253693)",55.641319,37.366429,район Ново-Переделкино,Краткое название: МФЦ района Ново-Переделкино<...,111000,259982,False
3,3066061,филиал Государственного бюджетного учреждения ...,7731419456,771001001,"Российская Федерация, город Москва, внутригоро...",Елисеева Елена Валерьевна,PublicPhone:(495) 777-77-77\n\n,13.06.2013,МФЦ района Академический,54,"(55.680726616647, 37.580068229737)",55.680727,37.580068,Академический район,Краткое название: МФЦ района Академический<br/...,162000,158460,True
4,3066062,филиал Государственного бюджетного учреждения ...,7731419456,771001001,"город Москва, Россошанская улица, дом 4, корпус 2",Горбачева Елена Анатольевна,PublicPhone:(495) 777-77-77\n\n,12.08.2013,МФЦ района Чертаново Южное,63,"(55.592468021305, 37.609509678775)",55.592468,37.609510,район Чертаново Южное,Краткое название: МФЦ района Чертаново Южное<b...,189000,123166,True


In [32]:
print('max:', delta_cnt_df['nearest_mfc'].max() / 1000, 'km. ' '\nmin:', delta_cnt_df['nearest_mfc'].min() / 1000 , 'km.')

max: 43.61748364582916 km. 
min: 0.02080932940071992 km.


In [33]:
model1 = 'mfc_chance_agreg'
delta_cnt_df[model1] = 0

#Настройка влияния весов на параметры плотности людей
alphas = {'home':1.0,'job':1.0,'day':1.0, 'move':1.0}
alphas_dlt = {'home':0.5,'job':0.5,'day':0.5, 'move':0.5}

tqdm.pandas(desc="calc mfc chance: aggregation model")

for feature in ['home', 'job', 'day', 'move']:
    delta_cnt_df[model1] = delta_cnt_df[model1] + alphas[feature] * delta_cnt_df[f'customers_cnt_{feature}']
for feature in ['home', 'job', 'day', 'move']:
    delta_cnt_df[model1] = delta_cnt_df[model1] + alphas_dlt[feature] *  delta_cnt_df[f'customers_dlt_{feature}']  
    
delta_cnt_df[model1] = delta_cnt_df[model1] + (delta_cnt_df['nearest_mfc'])# / 43617.48364582916)*1000
delta_cnt_df[model1] = delta_cnt_df[model1] + (delta_cnt_df['logistic'])

delta_cnt_df[model1] = delta_cnt_df[model1].progress_apply(lambda x: 1 + 10* x / 42070.344117)

C:\ProgramData\Anaconda3\lib\site-packages\tqdm\std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
calc mfc chance: aggregation model: 100%|████████████████████████████████████████████████████████████████████████| 10240/10240 [00:00<00:00, 787359.49it/s]


In [34]:
model2 = 'mfc_chance_balance'
def coeff_flow(percent):
    input_arr = [0.75,0.95,1.1,1.5,5]
    output_arr = [0.3,0.7,1.3,2,2]
    return np.interp(percent, input_arr, output_arr, left=0.0, right=2.0)

def coeff_distance(km_dist):
    input_arr = [2,3.5,5]
    output_arr = [0.3,0.7,1.0]
    return np.interp(km_dist, input_arr, output_arr, left=0.0,right=1.0)

def coeff_logistic(log_persent):
    input_arr = [0.5,1,1.5]
    output_arr = [0.3,0.5,1.0,]
    return np.interp(log_persent, input_arr, output_arr, left=0.0,right=1.0)
    
tqdm.pandas(desc="calc mfc chance: balance model")

#Расчёт необходимости исходя из текущей загруженности
delta_cnt_df[model2] = delta_cnt_df['nearest_mfc_id'].progress_apply(lambda x: coeff_flow(mfc_df.loc[mfc_df['global_id'] == x]['people_flow_rate'].values[0] / mfc_df.loc[mfc_df['global_id'] == x]['max_people_flow'].values[0]) )

#Расчёт необходимости исходя из будущей загруженности
delta_cnt_df[model2] = delta_cnt_df[model2] + 0.5 *   delta_cnt_df['nearest_mfc_id'].progress_apply(lambda x: coeff_flow(mfc_df.loc[mfc_df['global_id'] == x]['future_people_flow_rate'].values[0] / mfc_df.loc[mfc_df['global_id'] == x]['max_people_flow'].values[0]) )

#Расчёт необходимости исходя из удалённости 
delta_cnt_df[model2] = delta_cnt_df[model2] +  delta_cnt_df['nearest_mfc_distance'].progress_apply(lambda x: coeff_distance(x / 1000.0) )

#Расчёт необходимости исходя из логистики
delta_cnt_df[model2] = delta_cnt_df[model2] +  (delta_cnt_df['nearest_mfc_id'].progress_apply(lambda x: coeff_logistic(delta_cnt_df.loc[delta_cnt_df['nearest_mfc_id'] == x]['logistic'].mean())) /  delta_cnt_df['logistic']).apply(lambda x: coeff_logistic(x)) #


calc mfc chance: balance model: 100%|██████████████████████████████████████████████████████████████████████████████| 10240/10240 [00:08<00:00, 1144.57it/s]


In [53]:
delta_cnt_df.to_csv('june_full_data.csv')
delta_cnt_df.head()

,Unnamed: 0,Unnamed: 0.1,zid,customers_cnt_home,customers_cnt_job,customers_cnt_day,customers_cnt_move,lat,lon,customers_dlt_home,customers_dlt_job,customers_dlt_day,customers_dlt_move,nearest_mfc,logistic,mfc_chance,nearest_mfc_id,nearest_mfc_distance,metaInfo
0,0,0,32909,2,0,0,0,55.140287,37.033512,-2,0,0,0,43617.483646,0,11.367988,272644507,43617.483646,Насление: 2<br/><b>Прирост:</b> 2<br/><b>Логис...
1,1,1,32910,13,6,26,0,55.140316,37.041354,-12,-2,0,0,43407.995189,0,11.326988,272644507,43407.995189,Насление: 13<br/><b>Прирост:</b> 13<br/><b>Лог...
2,2,2,33261,2,0,2,0,55.144748,37.025618,0,0,0,0,43381.461499,0,11.312600,272644507,43381.461499,Насление: 2<br/><b>Прирост:</b> 2<br/><b>Логис...
3,3,3,33262,13,1,11,0,55.144778,37.033460,0,-1,0,0,43165.034942,2,11.266504,272644507,43165.034942,Насление: 13<br/><b>Прирост:</b> 13<br/><b>Лог...
4,4,4,33263,7,0,2,0,55.144808,37.041302,-3,0,-1,0,42953.339821,0,11.211549,272644507,42953.339821,Насление: 7<br/><b>Прирост:</b> 7<br/><b>Логис...
